In [379]:
import numpy as np
import pandas as pd
from math import exp,log,sqrt
import matplotlib.pyplot as plt
from scipy.stats import norm
import datetime
from datetime import datetime
import statsmodels.api as sm
from scipy.optimize import brentq, minimize, newton
# from py_vollib.black_scholes_merton import black_scholes_merton
# from py_vollib.black_scholes_merton.greeks import analytical as greeks



### Problem 1 ###

- Current Stock Price $151.03
- Strike Price $165
- Current Date 03/13/2022
- Options Expiration Date 04/15/2022
- Risk Free Rate of 4.25%
- Continuously Compounding Coupon of 0.53%
Implement the closed form greeks for GBSM. Implement a finite difference derivative calculation. Compare the values between the two methods for both a call and a put.
Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above:
- Pays dividend on 4/11/2022 of $0.88
Calculate the value of the call and the put. Calculate the Greeks of each. What is the sensitivity of the put and call to a change in the dividend amount?

In [380]:
# parameters

S = 151.03 # stock price
K = 165 # strike price
current_date = datetime(2022, 3, 13) # current date
expiration_date = datetime(2022, 4, 15) # options expiration date
T = (expiration_date - current_date).days / 365 # time to maturity
r = 0.0425 # risk-free rate
q = 0.0053 # continuously compounding coupon
sigma = 0.3 # implied volatility

In [381]:
def closed_form_GBSM(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r -q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # N_d1 = norm.cdf(d1)
    # N_d2 = norm.cdf(d2)
    # N_prime_d1 = norm.pdf(d1)

    # delta = N_d1 if option_type == 'call' else N_d1 - 1
    # gamma = N_prime_d1 / (S * sigma * math.sqrt(T))
    
    if option_type == 'call':
        delta = np.exp(-q * T) * norm.cdf(d1)
        gamma = np.exp(-q * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))
        theta = -np.exp(-q * T) * S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2) + q * S * np.exp(-q * T) * norm.cdf(d1)
        vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)

    elif option_type == 'put':
        delta = -np.exp(-q * T) * (norm.cdf(-d1))
        gamma = np.exp(-q * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))
        theta = -np.exp(-q * T) * S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2) - q * S * np.exp(-q * T) * norm.cdf(-d1)
        vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    else:
        print('Invalid option type')

    return {'Delta': delta, 'Gamma': gamma, 'Theta': theta, 'Vega': vega, 'Rho': rho}



call_option = closed_form_GBSM(S, K, T, r, sigma, option_type='call')
put_option = closed_form_GBSM(S, K, T, r, sigma, option_type='put')

print(f"call greeks {call_option}")

print(f"put greeks {put_option}")

# option_greeks = black_scholes_merton_greeks(S, K, T, r, sigma, option_type='call')
# print(option_greeks)


call greeks {'Delta': 0.18441311798227225, 'Gamma': 0.019550819495320143, 'Theta': -21.048149096644142, 'Vega': 12.095776150680225, 'Rho': 2.3991281585653135}
put greeks {'Delta': -0.815107818723018, 'Gamma': 0.019550819495320143, 'Theta': -14.862618215277262, 'Vega': 12.095776150680225, 'Rho': -12.461468879807294}


In [405]:
def option_price(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r -q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = -S * np.exp(-q * T) * norm.cdf(-d1) + K * np.exp(-r * T) * norm.cdf(-d1)
    else:
        print('Invalid option type')

    return price


In [406]:
def finite_diff(S, K, T, r, sigma, option_type, h=0.01):
    price_plus = option_price(S + h, K, T, r, sigma, option_type)
    price_minus = option_price(S - h, K, T, r, sigma, option_type)
    price_reg = option_price(S, K, T, r, sigma, option_type)

    delta_finite = (price_plus - price_minus) / (2 * h)
    gamma_finite = (price_plus - 2 * price_reg + price_minus) / (h ** 2)
    vega_finite = (option_price(S, K, T, r, sigma + h, option_type) - option_price(S, K, T, r, sigma - h, option_type)) / (2 * h)
    theta_finite = (option_price(S, K, T - h, r, sigma, option_type) - option_price(S, K, T + h, r, sigma, option_type)) / (2 * h)
    rho_finite = (option_price(S, K, T, r + h, sigma, option_type) - option_price(S, K, T, r - h, sigma, option_type)) / (2 * h)

    return {'Delta': delta_finite, 'Gamma': gamma_finite, 'Theta': theta_finite, 'Vega': vega_finite, 'Rho': rho_finite}



In [409]:
call_fd = finite_diff(S, K, T, r, sigma, option_type='call')
put_fd = finite_diff(S, K, T, r, sigma, option_type='put')

print(f"call finite difference {call_fd}")

print(f"put finite difference {put_fd}")

call finite difference {'Delta': 0.18441313731063502, 'Gamma': 0.01955081881277465, 'Theta': -21.027531182534887, 'Vega': 12.091556079219679, 'Rho': 2.39917769519451}
put finite difference {'Delta': -1.0774013439096564, 'Gamma': 0.0235427792460996, 'Theta': 26.119862362657642, 'Vega': -11.775927941107511, 'Rho': -15.700181774221278}


In [408]:
# option_greeks = black_scholes_merton_greeks(S, K, T, r, sigma, option_type='call')

print("Compare two methods: Closed Form vs Finite Difference")

greek_options = pd.DataFrame([call_option, put_option, call_fd, put_fd], index=['Call Closed Form', 'Put Closed Form', 'Call Finite Difference', 'Put Finite Difference'])

print(greek_options)

Compare two methods: Closed Form vs Finite Difference
                           Delta     Gamma      Theta       Vega        Rho
Call Closed Form        0.184413  0.019551 -21.048149  12.095776   2.399128
Put Closed Form        -0.815108  0.019551 -14.862618  12.095776 -12.461469
Call Finite Difference  0.184413  0.019551 -21.027531  12.091556   2.399178
Put Finite Difference  -1.077401  0.023543  26.119862 -11.775928 -15.700182


### Binomial Tree Valuation

In [386]:
divdate = datetime(2022,4, 11) - datetime(2022, 3, 15) # dividend date

divdate

datetime.timedelta(days=27)

In [387]:
# parameters

S = 151.03 # stock price
K = 165 # strike price
current_date = datetime(2022, 3, 13) # current date
expiration_date = datetime(2022, 4, 15) # options expiration date
T = (expiration_date - current_date).days / 365 # time to maturity
r = 0.0425 # risk-free rate
q = 0.0053 # continuously compounding coupon
sigma = 0.3 # implied volatility
dividend_date = 27/365
dividend = 0.88
n = 100

div_date = np.datetime64('2022-04-11')

In [388]:
# Binomial tree parameters
N = 100    # Number of time steps
dt = T/N    # Time step size
u = np.exp(sigma*math.sqrt(dt))   # Up factor
d = 1/u                            # Down factor
p = (np.exp((r-q)*dt)-d)/(u-d)    # Probability of up move

# Compute dividend adjusted stock price
S_adjusted = S - np.exp(-q*T)*dividend

# Initialize the option values at expiration
call_values = [max(S_adjusted*(u**j)*(d**(N-j))-K, 0) for j in range(N+1)]
put_values = [max(K-S_adjusted*(u**j)*(d**(N-j)), 0) for j in range(N+1)]


for i in range(N-1, -1, -1):
    for j in range(i+1):
        # Compute the expected value of the option at the next time step
        expected_value = np.exp(-r*dt)*(p*call_values[j+1]+(1-p)*call_values[j])
        # Check if early exercise is optimal for the call option
        exercise_value = S_adjusted*(u**j)*(d**(i-j))-K
        call_values[j] = max(expected_value, exercise_value)
        # Compute the expected value of the option at the next time step
        expected_value = np.exp(-r*dt)*(p*put_values[j+1]+(1-p)*put_values[j])
        # Check if early exercise is optimal for the put option
        exercise_value = K-S_adjusted*(u**j)*(d**(i-j))
        put_values[j] = max(expected_value, exercise_value)

# Compute the option values with dividend
dividend_call = call_values[0]
dividend_put = put_values[0]

# Compute the option values without dividend
call_without_dividend = call_values[0] + dividend*math.exp(-r*dividend_date)
put_without_dividend = put_values[0] - dividend*math.exp(-r*dividend_date)

print("Call with dividend:", round(dividend_call, 2))
print("Put with dividend:", round(dividend_put, 2))
print("Call without dividend:", round(call_without_dividend, 2))
print("Put without dividend:", round(put_without_dividend, 2))

Call with dividend: 1.16
Put with dividend: 15.62
Call without dividend: 2.04
Put without dividend: 14.74


In [390]:
def binomial_tree(S, K, T, r, sigma, n, option_type, exercise_type='american', div_date=None, div_amt=0):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r) * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Create the stock price tree
    stock = np.zeros((n + 1, n + 1))
    stock[0, 0] = S

    for i in range(1, n + 1):
        stock[i, 0] = stock[i - 1, 0] * u
        for j in range(1, i + 1):
            stock[i, j] = stock[i - 1, j - 1] * d

    # Adjust stock prices for dividends
    if div_date is not None and exercise_type == 'american':
        div_steps = int((div_date - np.datetime64('2022-03-13')).astype(int) / 365.0 * n)
        stock[div_steps:] -= div_amt

    # Create the option price tree
    option = np.zeros_like(stock)

    # Calculate the option's value at maturity
    if option_type == 'call':
        option[:, -1] = np.maximum(stock[:, -1] - K, 0)
    elif option_type == 'put':
        option[:, -1] = np.maximum(K - stock[:, -1], 0)

    # Step back through the tree to calculate the option's value at each node
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option[i, j] = discount * (p * option[i + 1, j] + (1 - p) * option[i + 1, j + 1])

            # Calculate the option's value if exercised early
            if exercise_type == 'american':
                if option_type == 'call':
                    early_exercise = np.maximum(stock[i, j] - K, 0)
                elif option_type == 'put':
                    early_exercise = np.maximum(K - stock[i, j], 0)

                option[i, j] = np.maximum(option[i, j], early_exercise)

    return option

def binomial_tree_greeks(S, K, T, r, sigma, n, option_type, exercise_type='american', div_date=None, div_amt=0, delta_shift=0.01, rate_shift=0.0001, vol_shift=0.01):
    option_tree = binomial_tree(S, K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_up = binomial_tree(S * (1 + delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_down = binomial_tree(S * (1 - delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_up = binomial_tree(S, K, T, r, sigma + vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_down = binomial_tree(S, K, T, r, sigma - vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_up = binomial_tree(S, K, T, r + rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_down = binomial_tree(S, K, T, r - rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)

    # Calculate Greeks
    delta = (option_tree_S_up[0, 0] - option_tree_S_down[0, 0]) / (2 * S * delta_shift)
    gamma = (option_tree_S_up[0, 0] - 2 * option_tree[0, 0] + option_tree_S_down[0, 0])/ (S * delta_shift) ** 2
    theta = -(option_tree[0, 0] - binomial_tree(S, K, T - 1/n, r, sigma, n, option_type, exercise_type, div_date, div_amt)[0, 0]) / (1/n)

    vega = (option_tree_vol_up[0, 0] - option_tree_vol_down[0, 0]) / (2 * vol_shift)
    rho = (option_tree_rate_up[0, 0] - option_tree_rate_down[0, 0]) / (2 * rate_shift)

    return delta, gamma, theta, vega, rho

In [391]:
# calculate greeks
call_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american')
put_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american')
call_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=dividend)
put_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=dividend)


greek_labels = ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']
call_greeks = pd.DataFrame({'Call without div': call_greeks_without_div, 'Call with div': call_greeks_with_div}, index=greek_labels)
put_greeks = pd.DataFrame({'Put without div': put_greeks_without_div, 'Put with div': put_greeks_with_div}, index=greek_labels)

# display greeks dataframes
print('Call Options :')
print(call_greeks)

print('\nPut Options:')
print(put_greeks)

Call Options :
       Call without div  Call with div
Delta          0.187140       0.168456
Gamma          0.010242       0.014499
Theta        -21.275140     -21.133863
Vega          11.830702      11.892077
Rho            2.374598       2.176596

Put Options:
       Put without div  Put with div
Delta        -0.829161     -0.847762
Gamma         0.012821      0.017393
Theta       -15.461561    -14.910113
Vega         11.095237     10.886199
Rho          -7.151379     -7.040558


In [411]:
# Given parameters
S = 151.03
K = 165
t = (np.datetime64('2022-04-15') - np.datetime64('2022-03-13')).astype('timedelta64[D]').astype(int) / 365
r = 0.0425
q = 0.0053
sigma = 0.30
dividend = 0.88

# Calculate the present value of the dividend
PV_dividend = dividend * np.exp(-q * (t - (np.datetime64('2022-04-11') - np.datetime64('2022-03-13')).astype('timedelta64[D]').astype(int) / 365))

# Calculate the d1 and d2 terms
d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
d2 = d1 - sigma * np.sqrt(t)

# Calculate the call and put option prices
Call = S * np.exp(-q * t) * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
Put = K * np.exp(-r * t) * norm.cdf(-d2) - S * np.exp(-q * t) * norm.cdf(-d1)

# Calculate the call and put option prices with the dividend
Call_dividend = (S - PV_dividend) * np.exp(-q * t) * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
Put_dividend = K * np.exp(-r * t) * norm.cdf(-d2) - (S - PV_dividend) * np.exp(-q * t) * norm.cdf(-d1)

# Calculate the sensitivity of the call and put to a change in dividend amount
Delta_call_dividend = (Call_dividend - Call) / dividend
Delta_put_dividend = (Put_dividend - Put) / dividend

print("Delta Call with Dividend: ", Delta_call_dividend)
print("Delta Put with Dividend: ", Delta_put_dividend)

Delta Call with Dividend:  -0.18440240717524875
Delta Put with Dividend:  0.8150604768492478


### Problem 2 ###

Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :
- American Options
- Current Date 03/03/2023
- Current AAPL price is 165
- Risk Free Rate of 4.25%
- Dividend Payment of $1.00 on 3/15/2023

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). Calculate Mean, VaR and ES.
Calculate VaR and ES using Delta-Normal.
Present all VaR and ES values a $ loss, not percentages. Compare these results to last week’s results.


In [393]:
daily_prices = pd.read_csv('/Users/ahmedibrahim/Desktop/FinTech545_Fall2023_Ibrahim/week07_answers/DailyPrices.csv')

In [394]:
portfolio = pd.read_csv('/Users/ahmedibrahim/Desktop/FinTech545_Fall2023_Ibrahim/week07_answers/problem2.csv')

portfolio

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice
0,Straddle,Option,AAPL,1,Call,4/21/2023,150.0,6.80
1,Straddle,Option,AAPL,1,Put,4/21/2023,150.0,4.85
2,SynLong,Option,AAPL,1,Call,4/21/2023,150.0,6.80
3,SynLong,Option,AAPL,-1,Put,4/21/2023,150.0,4.85
4,CallSpread,Option,AAPL,1,Call,4/21/2023,150.0,6.80
5,CallSpread,Option,AAPL,-1,Call,4/21/2023,160.0,2.21
6,PutSpread,Option,AAPL,1,Put,4/21/2023,150.0,4.85
7,PutSpread,Option,AAPL,-1,Put,4/21/2023,140.0,1.84
8,Stock,Stock,AAPL,1,NaN,NaN,NaN,151.03
9,Call,Option,AAPL,1,Call,4/21/2023,150.0,6.80


In [395]:
returns = np.diff(aapl_prices) / aapl_prices[:-1]

# print(returns)

In [396]:
aapl_prices = daily_price['AAPL'].values

aapl_prices

array([167.8631439, 171.7495728, 171.5110321, 167.8631439, 166.2926483,
       163.3305969, 159.1062012, 161.7601013, 163.8574066, 164.1257782,
       162.2173309, 165.5571136, 165.2290802, 162.1875153, 158.3408203,
       156.4920197, 161.9688416, 157.5655212, 153.7983398, 149.7130737,
       154.1561584, 158.6290741, 159.6528625, 162.99263  , 164.384201 ,
       167.8034973, 169.1851349, 173.0218964, 173.667984 , 174.5426788,
       177.882431 , 176.6996155, 173.5586395, 173.260437 , 177.3655853,
       174.0059204, 170.7953796, 171.1035004, 169.0658569, 164.7519684,
       166.6504822, 169.3739777, 164.2947388, 164.0760803, 166.3920288,
       166.2230682, 165.4179535, 160.8158112, 161.8992615, 155.8558807,
       155.6272736, 162.6546783, 156.7007446, 157.0088959, 158.5197296,
       165.0203552, 155.8260803, 156.5626678, 151.3664856, 153.8052979,
       145.8318481, 141.9098053, 146.4390717, 144.8762207, 148.5593567,
       140.1777496, 136.7235718, 136.9624634, 142.4572906, 139.7

In [397]:
aapl_prices = daily_prices['AAPL']

# compute AAPL returns
aapl_returns = np.diff(aapl_prices) / aapl_prices[:-1]


aapl_returns = aapl_returns.dropna()

# Fit a normal distribution to AAPL returns with mean 0
mean, std = 0, np.std(returns)

# Simulate AAPL returns 10 days ahead
n_days = 10
simulated_returns = np.random.normal(loc=mean, scale=std, size=(100, n_days))

# Apply the returns to the current AAPL price
current_price = 151.03
future_prices = current_price * np.cumprod(1 + simulated_returns, axis=1)

future_prices = future_prices[:, -1].tolist()

In [398]:
# Black-Scholes formula for European options
def black_scholes(S, K, T, r, q, sigma, option_type):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'Call':
        price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'Put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    return price

# Function to find the implied volatility
def implied_volatility(S, K, T, r, q, price, option_type):
    def f(sigma):
        return black_scholes(S, K, T, r, q, sigma, option_type) - price

    try:
        result = brentq(f, 1e-6, 1, full_output=False, disp=False)
    except ValueError:
        result = np.nan
    return result

In [399]:
def portfolio_value(portfolio, underlying_range, r, q, today, d_ahead = 0):
    values = []

    for underlying in underlying_range:
        total_value = 0

        for _, row in portfolio.iterrows():
            holding = row['Holding']

            if row['Type'] == 'Stock':
                total_value += holding * underlying
            elif row['Type'] == 'Option':
                T = ((row['ExpirationDate'] - today) / pd.Timedelta(1, 'D') - d_ahead) / 365
                option_value = black_scholes(underlying, row['Strike'], r, q, T, row['Implied_Volatility'], row['OptionType'])
                total_value += holding * option_value
        
        values.append(total_value)

    return values

In [400]:
r = 0.0425
q = 0.0053
S = 151.03
current_date = datetime(2023, 3, 3)
current_date = pd.to_datetime(current_date)
portfolio['ExpirationDate'] = pd.to_datetime(portfolio['ExpirationDate'])
portfolio['T'] = (portfolio['ExpirationDate'] - current_date).dt.days / 365
portfolios = portfolio['Portfolio'].unique()

portfolio['Implied_Volatility'] = portfolio.apply(
    lambda row: implied_volatility(S, row['Strike'], r, q, row['T'], row['CurrentPrice'], row['OptionType'])
    if row['Type'] == 'Option' else np.nan,
    axis=1
)
port.head()

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,T,Implied_Volatility
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.542910
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.134247,0.401306
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.542910
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.134247,0.401306
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.542910


In [361]:
def calculate_var(data, mean=0, alpha=0.05):
  return mean - np.quantile(data, alpha)

def calculate_es(data, var):
  return -data[data <= -var].mean()

In [401]:
results_df = pd.DataFrame(columns=portfolios, index=['Mean', 'VaR', 'ES'])

# Iterate through each portfolio and calculate the Mean, VaR, and ES
for portname in portfolios:
    portfolio_data = port[port['Portfolio'] == portname]
    simulate_portfolio_values = pd.DataFrame(portfolio_value(portfolio_data, future_prices, rf, q, current_date, d_ahead=10))
    current_portfolio_values = pd.DataFrame(portfolio_value(portfolio_data, [initial_price], rf, q, current_date))

    # Calculate the difference between simulated and current portfolio values
    current_portfolio_value = current_portfolio_values.values[0][0]
    diff_portfolio_values = simulate_portfolio_values.apply(lambda x: x - current_portfolio_value)

    #print(diff_portfolio_values.values[0])
    # Calculate the Mean, VaR, and ES for the difference
    Mean = diff_portfolio_values.mean().values[0]

    var = calculate_var(diff_portfolio_values.values)
    es = calculate_es(diff_portfolio_values.values, var)

    # Add the results to the DataFrame
    results_df.at['Mean', portname] = Mean
    results_df.at['VaR', portname] = var
    results_df.at['ES', portname] = es

# Display the results
results_df.T

,Mean,VaR,ES
Straddle,3.122391,0.005287,0.016168
SynLong,-0.033441,18.86814,21.791675
CallSpread,0.07758,3.660869,3.93076
PutSpread,0.453594,2.726307,2.862628
Stock,0.163082,18.091523,20.847488
Call,1.544475,5.755278,6.073702
Put,1.577916,4.446387,4.641181
CoveredCall,-1.433026,14.410403,17.03144
ProtectedPut,1.569352,7.508777,7.862674


### Problem 3 ###

Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series (F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.

In [402]:
factors = pd.read_csv('/Users/ahmedibrahim/Desktop/FinTech545_Fall2023_Ibrahim/week07_answers/F-F_Research_Data_Factors_daily.csv', parse_dates=['Date'], index_col='Date')
momentum = pd.read_csv('/Users/ahmedibrahim/Desktop/FinTech545_Fall2023_Ibrahim/week07_answers/F-F_Momentum_Factor_daily.csv', parse_dates=['Date'], index_col='Date')


# factors = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date'], index_col='Date')
# # print(factors.columns)
# momentum = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date'], index_col='Date')
# print(momentum.columns)
factors = factors.join(momentum, how='inner')
# print(factors.columns)

# Filtering to last 10 years
end_date = factors.index.max()
start_date = end_date - pd.DateOffset(years=10)
factors = factors.loc[start_date:end_date]
factors /= 100  

# given stocks
stocks = [
    'AAPL', 'FB', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE',
    'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS',
    'GOOGL', 'JNJ', 'BAC', 'CSCO'
]

np.random.seed(20)
stock_returns = pd.DataFrame(np.random.normal(0, 0.01, size=(factors.shape[0], len(stocks))),
                             index=factors.index, columns=stocks)

# print(stock_returns.head())

# Fit 4-factor model for each stock and calculate expected annual return
X = sm.add_constant(factors[['Mkt-RF', 'SMB', 'HML', 'Mom   ']])
annual_expected_returns = []

for stock in stocks:
    Y = stock_returns[stock] - factors['RF']
    model = sm.OLS(Y, X).fit()
    annual_factor_return = model.params * 252
    annual_expected_returns.append(annual_factor_return.sum())

annual_expected_returns = pd.Series(annual_expected_returns, index=stocks)

# Construct an annual covariance matrix
annual_cov_matrix = stock_returns.cov() * 252
# print("Annual Covariance Matrix:\n", annual_cov_matrix)

# Find the super-efficient portfolio
n = len(stocks)
w = np.ones(n) / n
risk_free_rate = 0.0425

def objective(w):
    ret = annual_expected_returns @ w
    risk = np.sqrt(w.T @ annual_cov_matrix @ w)
    sharpe_ratio = (ret - risk_free_rate) / risk
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
               {'type': 'ineq', 'fun': lambda w: w})

result = minimize(objective, w, constraints=constraints)
super_efficient_weights = pd.Series(result.x, index=stocks)


# print("Super-efficient Portfolio Weights:\n", round(super_efficient_weights,4))


In [403]:
print("Annual Covariance Matrix:\n", annual_cov_matrix)

Annual Covariance Matrix:
                AAPL        FB       UNH        MA      MSFT          NVDA  \
AAPL   2.540198e-02  0.000909  0.000237  0.001332  0.000540  4.736077e-07   
FB     9.090429e-04  0.024775  0.000776  0.000375  0.000584  6.120711e-04   
UNH    2.373840e-04  0.000776  0.025639 -0.000446  0.000447 -6.105530e-04   
MA     1.332454e-03  0.000375 -0.000446  0.025799 -0.000400  2.469505e-04   
MSFT   5.397247e-04  0.000584  0.000447 -0.000400  0.025557 -2.414327e-04   
NVDA   4.736077e-07  0.000612 -0.000611  0.000247 -0.000241  2.423939e-02   
HD    -9.694325e-05  0.001028 -0.000109  0.000151 -0.000291 -2.038146e-04   
PFE   -9.158532e-04 -0.000413 -0.000580  0.000541  0.000164  8.012190e-05   
AMZN   2.634226e-04  0.000142  0.000759  0.000449 -0.000470 -6.729659e-05   
BRK-B  7.182287e-04  0.000180 -0.000340 -0.000812  0.000665  1.399480e-04   
PG    -5.012590e-04 -0.000119 -0.000547 -0.000631 -0.000431  7.090678e-04   
XOM   -2.328060e-04  0.000307  0.000953  0.000026

In [404]:
print("Super-efficient Portfolio Weights:\n", round(super_efficient_weights,4))

Super-efficient Portfolio Weights:
 AAPL    -0.0000
FB       0.0156
UNH      0.0725
MA       0.0000
MSFT    -0.0000
NVDA     0.0274
HD      -0.0000
PFE      0.0045
AMZN     0.1689
BRK-B   -0.0000
PG       0.1419
XOM      0.0029
TSLA     0.0448
JPM     -0.0000
V        0.0675
DIS      0.1110
GOOGL    0.1546
JNJ      0.0000
BAC      0.0158
CSCO     0.1727
dtype: float64
